# Camera Aruco Tag Distance Test

Use this notebook to test how far away a standard USB camera (not the Oak-d, which requires seperate software) can detect an Aruco Tag. With a standard high definition camera, we were able to detect a tag 30-35 feet away. The competition may place the tag up to 67 feet away, so to be safe, we want to reliably detect a tag from 70 feet.

## Setup

You will need to install OpenCV-Contrib version 4.6.0.66 (a lower version is okay, but a higher has breaking API changes).

In [1]:
import cv2
import numpy as np
from collections import deque
import os
import glob

In [2]:
# load the 4x4_50 aruco tag dictionary
arucoDict = cv2.aruco.Dictionary_get(cv2.aruco.DICT_4X4_50)
parameters = cv2.aruco.DetectorParameters_create()

## Camera

Here is where we set up the camera. Change the `CAMERA_NUMBER`, `FRAME_WIDTH`, and `FRAME_HEIGHT` variables to whatever camera you are testing.

**!!IMPORTANT!!** Look at the output of this cell. If it is `True` then the camera was opened correctly and the frame width/height was properly set. If it is `False` then something went wrong and the camera closes.

In [5]:
CAMERA_NUMBER = 0
FRAME_WIDTH = 1920
FRAME_HEIGHT = 1080

cap = cv2.VideoCapture(CAMERA_NUMBER)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)


False

## Detecting Aruco Tags

This is the main loop where we try to find aruco tags. It'll draw a green bounding box around any tag it detects. It also looks through the last 10 frames to see if it can detect a tag in any of those frames. This helps its accuracy at long distances because the odds of it detecting a tag in single frame isn't great, but over many frames, it's pretty decent. Press 'q' to stop the loop.

NOTE: if you just hit `Run All` and then stop the loop with `q`, it will also close the camera and you will have to re-open it before you can start the loop again.

In [4]:
frame_queue = deque(maxlen=10)

while True:
    # read the frame
    ret, frame = cap.read()
    frame_queue.append(frame)

    corners, ids = [], []

    for test_frame in frame_queue:
        # detect the aruco markers in the image
        possible_corners, possible_ids, rejectedImgPoints = cv2.aruco.detectMarkers(test_frame, arucoDict,parameters=parameters)
        if len(possible_corners) > 0:
            corners.extend(possible_corners)
            ids.append(possible_ids.flatten())
            break

    for corner, tag_id in zip(corners, ids):
        # loop over the detected ArUCo corners
        #for (markerCorner, markerID) in zip(corners, ids):
        # extract the marker corners (which are always returned in
        # top-left, top-right, bottom-right, and bottom-left order)
        # corners = markerCorner.reshape((4, 2))
        tag_id = tag_id[0]

        (topLeft, topRight, bottomRight, bottomLeft) = corner[0]
        # convert each of the (x, y)-coordinate pairs to integers
        topRight = (int(topRight[0]), int(topRight[1]))
        bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
        bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
        topLeft = (int(topLeft[0]), int(topLeft[1]))
        # draw the bounding box of the ArUCo detection
        cv2.line(frame, topLeft, topRight, (0, 255, 0), 2)
        cv2.line(frame, topRight, bottomRight, (0, 255, 0), 2)
        cv2.line(frame, bottomRight, bottomLeft, (0, 255, 0), 2)
        cv2.line(frame, bottomLeft, topLeft, (0, 255, 0), 2)
        # compute and draw the center (x, y)-coordinates of the ArUco
        # marker
        cX = int((topLeft[0] + bottomRight[0]) / 2.0)
        cY = int((topLeft[1] + bottomRight[1]) / 2.0)
        cv2.circle(frame, (cX, cY), 4, (0, 0, 255), -1)
        # draw the ArUco marker ID on the frame
        cv2.putText(frame, str(tag_id),
            (topLeft[0], topLeft[1] - 15), cv2.FONT_HERSHEY_SIMPLEX,
            0.5, (0, 255, 0), 2)


    frame = cv2.resize(frame, None, fx=.5, fy=.5)
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break

error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\aruco\src\aruco.cpp:1097: error: (-215:Assertion failed) !_image.empty() in function 'cv::aruco::detectMarkers'


In [ ]:
# close the camera
cap.release()
# close all windows
cv2.destroyAllWindows()